In [1]:
import numpy as np
import cv2

In [8]:
VIDEO_PATH='./armor.mp4'
OUTPUT_PATH='./output.mp4'

In [110]:
cap=cv2.VideoCapture(VIDEO_PATH)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
out = cv2.VideoWriter(OUTPUT_PATH, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

#hsv下的颜色掩膜，筛选出橙黄色灯条
lower_msk=np.array([10, 100, 100])
upper_msk=np.array([40, 255, 255])

while True:
    ret, frame = cap.read()
    if not ret:
        break

    #二值化保留较亮部分，之后筛选颜色
    _, thresh = cv2.threshold(frame, 230, 255, cv2.THRESH_BINARY)
    hsv = cv2.cvtColor(thresh, cv2.COLOR_BGR2HSV)
    msk=cv2.inRange(hsv, lower_msk, upper_msk)

    #检测轮廓
    contours, _ = cv2.findContours(msk, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    lightbars = []

    colors=[[255,150,150],[150,255,150],[150,150,255]]
    i=0
    #筛选轮廓，加入灯条列表
    for contour in contours:
            if cv2.contourArea(contour) > 100:  
                x,y,w,h = cv2.boundingRect(contour)
                if(w>50 or h<20):
                     continue
                cv2.drawContours(frame, [contour], -1, colors[i], 2)
                i=(i+1)%3
                lightbars.append(contour)

    def dist(X,Y):
        return np.sum((X-Y)**2)**0.5

    def findcentre(contour):
        x,y,w,h = cv2.boundingRect(contour)
        return np.array([x+w//2, y+h//2])

    def isadja(light1, light2):
        C1, C2 = findcentre(light1), findcentre(light2)
        return abs(C1[1]-C2[1])<20 and abs(C1[0]-C2[0])<100

    #相邻的灯条构成装甲板，在图中绘制出矩形和中心
    for i in range(1,len(lightbars)):
        light1, light2 = lightbars[i-1], lightbars[i]
        x1, y1, w1, h1 = cv2.boundingRect(light1)
        x2, y2, w2, h2 = cv2.boundingRect(light2)

        if (isadja(light1, light2)):
            mn = np.array([min(x1, x2), min(y1, y2)])
            mx = np.array([max(x1+w1, x2+w2), max(y1+h1, y2+h2)])
            cen= (mn+mx)//2
            cv2.rectangle(frame, mn, mx,(0, 255, 0), 2)
            cv2.circle(frame, cen, 2, (255, 0, 0), 2)
         

    out.write(frame)

    cv2.imshow('Detection', frame)
    if cv2.waitKey(20) == ord(' '): 
        break

cap.release()
out.release()
cv2.destroyAllWindows()

In [83]:
{}

{}